# 🧪 Decoherence Audio Effect
In a similar way to the decoherent image effect, you can apply decoherence to the audio processing. Adding coherence arbitrary level 3 gives an astounding effect that truly resembles the visual analogue. However, the computation of the density matrix take much longer that the noiseless ideal statevector, so, if it was already a lengthy process, now it could take you up to several hours to produce one audio file at a given decoherence level. So for the sake of this example I suggest slicing the sample to 20 instead of the >3k samples and will only take about 10 minutes but you can here the main musical motive of "underground". A sample of the first 3 seconds with decoherence=3 is provided.

### Prepare Noisy Backend and Sims
You can either use backend data directly from IBMQiskit service or use the noise model we included here if you do not have an account and token

In [ ]:
import QPIXL.helper as hlp
from QPIXL.qiskit.qpixl import cFRQI
from QPIXL.qiskit.qpixl_angs import cFRQIangs, decodeAngQPIXL
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
import soundfile
from itertools import chain

warnings.filterwarnings("ignore")

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

use_remote = False
noisy_backend = None

if use_remote:
    service = QiskitRuntimeService(
        channel="ibm_quantum",
        token="...", # add your IBMQuantum token here
    )
    backend = service.backend("ibm_brisbane")
    noise_model = NoiseModel.from_backend(backend)
    noisy_simulator = AerSimulator(method='density_matrix', noise_model=noise_model)
else:
    # If you are unwillig to provide credentials or don't have an account
    # a object snapshot is included.
    # You may need to change the path to point to local absolute path
    # as path resolution of curren notebook is unpredictable in Jupyter
    T1s = [
        0.00025172438284199106, 
        0.0003629012969698477, 
        0.0003598508401631463, 
        0.00039118816145153253, 
        0.00023875650712608485, 
        0.0002704203434010988, 
        0.00035383501950940493, 
        0.000300554264232933, 
        0.0002797320000067376, 
        0.00010722972896239507, 
        0.0003607083147081156, 
        0.00016270939235588344
    ]
    from pathlib import Path
    current_dir = Path().resolve()
    file_path = os.path.join(current_dir, 'QuantumArtHack/noise_models/ibm_brisbane.pkl')
    print(file_path)
    with open(file_path, 'rb') as f:
        import pickle
        loaded_noise_model = pickle.load(f)
    noisy_simulator = AerSimulator(method='density_matrix', noise_model=loaded_noise_model)
    

/home/lsjcp/py/QuantumArtHack/noise_models/ibm_brisbane.pkl


Now, after defining the backend and simulator with realistic noise model, let's apply the decoherence effect to the audio or a part of it

In [ ]:
PROCESS_BATCH_SIZE = 2  # Set to your desired batch size but beware of memory limits

def process_audio_noisy(
    input_file,
    output_dir,
    pattern_func=None,
    pattern2_func=None,
    post_process_func=None,
    tag="",
    compression=0,
    max_samples=None,
):
    import os
    from itertools import chain
    import numpy as np
    import soundfile
    from qiskit.quantum_info import DensityMatrix

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    file_name, _ = os.path.splitext(os.path.basename(input_file))
    metadata_file_path = os.path.join(
        output_dir, f"{file_name}_metadata_{tag}_c{compression}.txt"
    )
    if not os.path.exists(metadata_file_path):
        data, samplerate = soundfile.read(input_file)
        chunk_size = 512
        sections = [
            hlp.pad_0(data[i : i + chunk_size]) for i in range(0, len(data), chunk_size)
        ]
        if max_samples:
            sections = sections[:max_samples]
        else:
            sections = sections[:-1]

        decoded = []
        circuits = []
        chunk_info = []

        # Prepare circuits
        for index, section in enumerate(sections):
            normalized_section = section - np.min(section)
            patt1, patt2 = None, None
            if pattern_func:
                patt1, _ = pattern_func
            if pattern2_func:
                patt2, _ = pattern2_func
            qc = cFRQIangs(normalized_section, compression, patt1, patt2)
            if post_process_func is not None:
                post_process_func(qc)
            qc.save_density_matrix()
            circuits.append(qc)
            chunk_info.append((index, section))

        # Run circuits in batches
        for i in range(0, len(circuits), PROCESS_BATCH_SIZE):
            batch = circuits[i:i+PROCESS_BATCH_SIZE]
            batch_info = chunk_info[i:i+PROCESS_BATCH_SIZE]
            print(f"Processing {file_name} batch: chunks {i+1}-{i+len(batch)} of {len(circuits)}", end="\r")

            job = noisy_simulator.run(batch)
            result = job.result()

            for j, (index, section) in enumerate(batch_info):
                rho = result.data(j)['density_matrix']
                state_vector = np.real(np.diag(rho))
                decoded.append(
                    decodeAngQPIXL(
                        state=state_vector,
                        qc=batch[j],
                        trace=2,
                        max_pixel_val=section.max(),
                        min_pixel_val=section.min(),
                    )
                )

        decoded_full = np.array(list(chain.from_iterable(decoded)))
        soundfile.write(
            os.path.join(output_dir, f"{file_name}_output_{tag}_c{compression}.wav"),
            decoded_full,
            samplerate,
        )



def pattern(angle):
    """Example pattern function that applies a
    CRX gate to the quantum circuit in the algortihm register.
    Returns a function that takes a circuit and applies the pattern.
    """

    def pattern(circ):
        circ.crx(angle, 1, 0)

    return pattern, angle


def pattern2(angle):
    def pattern(circ):
        circ.cry(angle, 1, 0)

    return pattern, angle


def post(decoherence, dividend, steps):
    def post(circ):
        for _ in range (0, decoherence):
            for q in range(circ.num_qubits):
                t1 = backend.properties().t1(q) if use_remote else T1s[q]
                delay_ns = int(t1 * 1e9 / dividend) / steps
                circ.delay(delay_ns, q, 'ns')
            circ.barrier()
    return post

In [ ]:
BASE_DIR = "QuantumArtHack/Sample_Material"

DIVIDEND = 5 # How much the T1 used as reference is divided by to get the delay time
STEPS = 20 # Number of steps to divide the decoherence time into

decoherence_level = 8 # How much decoherence to apply, 0 means no decoherence
file_name = "underground.mp3" # Your file

file = os.path.join(BASE_DIR, file_name)
input_file = os.path.join(BASE_DIR, file_name)
output_dir = os.path.join(BASE_DIR, os.path.splitext(file)[0])
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

process_audio_noisy(
    input_file,
    output_dir,
    pattern_func=None,
    pattern2_func=None,
    post_process_func=post(
        decoherence=decoherence_level, 
        dividend=DIVIDEND, 
        steps=STEPS
    ),
    compression=90,
    tag=f"decoherence_{decoherence_level}",
    max_samples=100,  # Limit the number of samples to process for demonstration, none for full
) 

print(f"Processed {file_name} with decoherence level {decoherence_level} and saved to {output_dir}")